# Demo 1: JDBC Catalog with Apache Iceberg

This lesson demonstrates using **JDBC Catalog** with Apache Iceberg.

**Architecture:**
- Spark (Query Engine)
- PostgreSQL (JDBC Catalog)
- MinIO (S3-compatible storage)

**Benefits:**
- ✅ Production-ready
- ✅ Simple setup
- ✅ ACID guarantees
- ✅ Easy migration to S3

## 1. Configure Spark with JDBC Catalog

In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

# Configuration
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
MINIO_HOST = "minio"  # Use Docker service name (resolved via internal DNS)
WAREHOUSE_PATH = "s3a://warehouse"

# PostgreSQL (JDBC Catalog) configuration
POSTGRES_USER = "iceberg"
POSTGRES_PASSWORD = "iceberg"
POSTGRES_DB = "iceberg"
POSTGRES_HOST = "postgres"
POSTGRES_PORT = "5432"

conf = (
    pyspark.SparkConf()
        .setAppName('iceberg_jdbc_catalog')
        .set('spark.jars.packages', 
             'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
             'software.amazon.awssdk:bundle:2.20.26,'
             'software.amazon.awssdk:url-connection-client:2.20.26,'
             'org.postgresql:postgresql:42.7.1')
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        
        # JDBC Catalog Configuration
        .set('spark.sql.catalog.spark_catalog', 
             'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.spark_catalog.catalog-impl', 
             'org.apache.iceberg.jdbc.JdbcCatalog')
        .set('spark.sql.catalog.spark_catalog.uri', 
             f'jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')
        .set('spark.sql.catalog.spark_catalog.jdbc.user', POSTGRES_USER)
        .set('spark.sql.catalog.spark_catalog.jdbc.password', POSTGRES_PASSWORD)
        .set('spark.sql.catalog.spark_catalog.warehouse', WAREHOUSE_PATH)
        .set('spark.sql.catalog.spark_catalog.io-impl', 
             'org.apache.iceberg.aws.s3.S3FileIO')
        
        # MinIO S3 Configuration for Iceberg S3FileIO (AWS SDK v2)
        .set('spark.sql.catalog.spark_catalog.s3.endpoint', f'http://{MINIO_HOST}:9000')
        .set('spark.sql.catalog.spark_catalog.s3.path-style-access', 'true')
        .set('spark.sql.catalog.spark_catalog.s3.access-key-id', MINIO_ACCESS_KEY)
        .set('spark.sql.catalog.spark_catalog.s3.secret-access-key', MINIO_SECRET_KEY)
        
        # MinIO S3 Configuration for Hadoop s3a filesystem (if needed)
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
        .set('spark.hadoop.fs.s3a.endpoint', f'http://{MINIO_HOST}:9000')
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running with Iceberg and JDBC Catalog")
print(f"Catalog: JDBC (PostgreSQL at {POSTGRES_HOST}:{POSTGRES_PORT})")
print(f"Storage: MinIO at {MINIO_HOST}:9000")
print(f"Warehouse: {WAREHOUSE_PATH}")

Spark Running with Iceberg and JDBC Catalog
Catalog: JDBC (PostgreSQL at postgres:5432)
Storage: MinIO at minio:9000
Warehouse: s3a://warehouse


## 2. Create Default Namespace

JDBC Catalog starts empty - we need to create the `default` namespace first.

In [2]:
# Create default namespace if it doesn't exist
spark.sql("CREATE NAMESPACE IF NOT EXISTS spark_catalog.default")
print("✓ Created default namespace")

✓ Created default namespace


## 3. Verify Catalog Connection

In [3]:
# Show available databases
spark.sql("SHOW DATABASES").show()

# Show existing tables
spark.sql("SHOW TABLES IN spark_catalog.default").show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



## 4. Load CSV Data

In [4]:
# Load CSV into temporary view
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2011.csv")
csv_df.createOrReplaceTempView("csv_open_2011")

print(f"Loaded {csv_df.count()} rows from CSV")
csv_df.show(5)

Loaded 13126 rows from CSV
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|         regionName|affiliateId|      affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|       47661|     Dan Bailey|      Dan|  Bailey|   ACT|     M|               NULL|               NULL|       6|       Central East|          0|    CrossFit Legacy| 27|  NULL|  NULL|          1|          43|       1|2011|
|      124483| Joshua Bridges|   Joshua| Bridges|   ACT|     M|               NULL|  

## 4a. Create Iceberg Table

In [5]:
# Create Iceberg table from CSV data
spark.sql("""
    CREATE TABLE IF NOT EXISTS spark_catalog.default.df_open_2011_jdbc 
    USING iceberg 
    AS SELECT * FROM csv_open_2011
""")

print("✓ Created Iceberg table: spark_catalog.default.df_open_2011_jdbc")

✓ Created Iceberg table: spark_catalog.default.df_open_2011_jdbc


## 5. Query Iceberg Table

In [6]:
# Query the table
spark.sql("SELECT * FROM spark_catalog.default.df_open_2011_jdbc LIMIT 10").show()

# Count records
spark.sql("SELECT COUNT(*) as total FROM spark_catalog.default.df_open_2011_jdbc").show()

+------------+--------------------+---------+--------------------+------+------+-------------------+-------------------+--------+-------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|competitorId|      competitorName|firstName|            lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|         regionName|affiliateId|      affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+--------------------+---------+--------------------+------+------+-------------------+-------------------+--------+-------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|       47661|          Dan Bailey|      Dan|              Bailey|   ACT|     M|               NULL|               NULL|       6|       Central East|          0|    CrossFit Legacy| 27|  NULL|  NULL|          1|          43|       1|2011|
|      124483|      Joshua Bridges|   Joshua

## 6. Uploading multiple Dataframes and Adding to Iceberg

In [7]:
# Load CSV into temporary view
df_2011 = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2011.csv")
df_2012 = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2012.csv")
df_2013 = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2013.csv")


df_2011.writeTo("spark_catalog.default.multi_year_iceberg") \
    .using("iceberg") \
    .create()

# 2. Add the next DataFrames
df_2012.writeTo("spark_catalog.default.multi_year_iceberg").append()
df_2013.writeTo("spark_catalog.default.multi_year_iceberg").append()



Note the two different ways we added Iceberg tables.

In **4a.** we added an Iceberg table using an SQL statement something with CTAS (Create Table as SELECT) using `spark.sql()`

In **6.** we used the `writeTo()` API which is a more python-like approach to build a table. Notice we also appended the data of 2012 and 2013 into the Iceberg Table. 

### Now we can query the entire table

In [8]:
spark.sql("SELECT * FROM spark_catalog.default.multi_year_iceberg LIMIT 10").show()

spark.sql("SELECT COUNT(*) as total FROM spark_catalog.default.multi_year_iceberg").show()

+------------+----------------+---------+--------+------+------+-------------------+-------------------+--------+-------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|  competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|         regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+----------------+---------+--------+------+------+-------------------+-------------------+--------+-------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|       11435|    Rich Froning|     Rich| Froning|   ACT|     M|               NULL|               NULL|       6|       Central East|       3220|     CrossFit Mayhem| 25|  NULL|  NULL|          1|          28|       1|2013|
|      151906|      Mikko Salo|    Mikko|    Salo|   ACT|     M|               NULL|               NULL|

## 7. ACID Operations - Delete Records

In [9]:
# Delete records (ACID transaction)
spark.sql("""
    DELETE FROM spark_catalog.default.df_open_2011_jdbc 
    WHERE gender = 'M'
""")

print("✓ Deleted records")

# Verify deletion
spark.sql("SELECT COUNT(*) as total FROM spark_catalog.default.df_open_2011_jdbc").show()

✓ Deleted records
+-----+
|total|
+-----+
| 4506|
+-----+



## 8. Time Travel & History

In [10]:
# Show table history
spark.sql("SELECT * FROM spark_catalog.default.df_open_2011_jdbc.history").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2026-01-15 18:09:12.043|4644534369010550327|NULL               |true               |
|2026-01-15 18:09:14.69 |6346540157899538446|4644534369010550327|true               |
+-----------------------+-------------------+-------------------+-------------------+



In [11]:
# Show snapshots
spark.sql("SELECT snapshot_id, parent_id, operation FROM spark_catalog.default.df_open_2011_jdbc.snapshots").show(truncate=False)

+-------------------+-------------------+---------+
|snapshot_id        |parent_id          |operation|
+-------------------+-------------------+---------+
|4644534369010550327|NULL               |append   |
|6346540157899538446|4644534369010550327|overwrite|
+-------------------+-------------------+---------+



In [42]:
# 1. Run the query to get the 2 most recent snapshots
# Row 0 = Current (Latest), Row 1 = Previous
history_df = spark.sql("""
    SELECT snapshot_id
    FROM spark_catalog.default.df_open_2011_jdbc.snapshots 
    ORDER BY committed_at DESC 
    LIMIT 2
""")

# 2. Collect the data to the driver as a list of Row objects
rows = history_df.collect()

# 3. Access the data using list indexing
# Make sure we have at least 2 rows (a history)
if len(rows) >= 2:
    current_id = rows[0][0]  # First row, First column (Latest)
    previous_id = rows[1][0] # Second row, First column (Previous)

    print(f"Current ID: {current_id}")
    print(f"Previous ID: {previous_id}")

    # 4. Query using the previous ID
    spark.sql(f"""
        SELECT COUNT(*) as count_previous 
        FROM spark_catalog.default.df_open_2011_jdbc 
        VERSION AS OF {previous_id}
    """).show()

    spark.sql(f"""
        SELECT COUNT(*) as count_current 
        FROM spark_catalog.default.df_open_2011_jdbc 
        VERSION AS OF {current_id}
    """).show()
    
else:
    print("Not enough history to find a previous version.")

Current ID: 6346540157899538446
Previous ID: 4644534369010550327
+--------------+
|count_previous|
+--------------+
|         13126|
+--------------+

+-------------+
|count_current|
+-------------+
|         4506|
+-------------+



## 9. Table Metadata

In [17]:
# Show table schema
spark.sql("DESCRIBE spark_catalog.default.df_open_2011_jdbc").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|       competitorId|   string|   NULL|
|     competitorName|   string|   NULL|
|          firstName|   string|   NULL|
|           lastName|   string|   NULL|
|             status|   string|   NULL|
|             gender|   string|   NULL|
|countryOfOriginCode|   string|   NULL|
|countryOfOriginName|   string|   NULL|
|           regionId|   string|   NULL|
|         regionName|   string|   NULL|
|        affiliateId|   string|   NULL|
|      affiliateName|   string|   NULL|
|                age|   string|   NULL|
|             height|   string|   NULL|
|             weight|   string|   NULL|
|        overallRank|   string|   NULL|
|       overallScore|   string|   NULL|
|           genderId|   string|   NULL|
|               year|   string|   NULL|
+-------------------+---------+-------+

